# 🧠 Automatic Tumor Detection System
## BME 271D Final Project - Ege, Max, Sasha

### Using Frequency-Domain Analysis for Medical Image Segmentation

**What this tool does:**
1. Analyzes medical images (MRI/CT scans) using FFT
2. Applies frequency-domain filters to enhance tumor features
3. **Automatically detects if a tumor is present**
4. Filters out normal tissue boundaries and image artifacts
5. Provides confidence scores and tumor localization

**Clinical Applications:**
- Brain tumor detection (gliomas, meningiomas)
- Soft tissue sarcoma screening
- Liver/kidney lesion identification

In [ ]:
# ========== SETUP ==========
!pip install -q numpy matplotlib scipy scikit-image pandas

# Download our code from GitHub
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/generate_realistic_tumors.py

# Download sample tumor images
!mkdir -p data/images data/masks
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_001.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_002.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_003.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_001.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_002.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_003.png

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, disk, erosion
from skimage.segmentation import clear_border

# ========== HELPER FUNCTIONS ==========
def load_image_safe(filepath):
    """Load any image and convert to grayscale float [0,1]"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img).astype(np.float64) / 255.0
    return arr

def load_mask_safe(filepath):
    """Load any image and convert to binary mask"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img)
    return (arr > 127).astype(np.uint8)

def remove_border_regions(mask, border_size=10):
    """
    Remove any regions that touch the image border.
    This eliminates false positives from image edges and normal tissue boundaries.
    """
    # Clear anything touching the border
    cleared = clear_border(mask, buffer_size=border_size)
    return cleared

def is_normal_tissue_boundary(region, image):
    """
    Check if a region is likely a normal tissue boundary (muscle, fat interface)
    rather than a tumor.
    
    Normal tissue boundaries are:
    - Very elongated (high eccentricity)
    - Thin/arc-shaped
    - Along the perimeter of organs
    """
    # Check eccentricity (0 = circle, 1 = line)
    if region.eccentricity > 0.95:  # Very elongated
        return True
    
    # Check if it's a thin arc (small area but long perimeter)
    compactness = (region.perimeter ** 2) / (region.area + 1e-8)
    if compactness > 50:  # Very thin structure
        return True
    
    # Check if region is mostly on the edge of bright areas (boundary)
    bbox = region.bbox
    roi = image[bbox[0]:bbox[2], bbox[1]:bbox[3]]
    
    return False

# ========== IMPROVED TUMOR DETECTION ==========
def detect_tumor_regions(image, sensitivity=0.5):
    """
    Detect actual tumor regions while filtering out:
    - Image borders and artifacts
    - Normal tissue boundaries (muscle/fat interfaces)
    - Small noise regions
    """
    h, w = image.shape
    total_pixels = h * w
    
    # Stricter size constraints
    min_region_size = int(total_pixels * 0.005)  # 0.5% minimum
    max_region_size = int(total_pixels * 0.25)   # 25% maximum (tumors shouldn't be huge)
    
    results = {}
    
    # ===== METHOD 1: Intensity-based detection =====
    # Look for abnormally bright regions (common in tumors)
    smoothed = ndimage.gaussian_filter(image, sigma=2)
    
    mean_val = smoothed.mean()
    std_val = smoothed.std()
    
    # Adaptive threshold based on image statistics
    # Tumors are typically >2 std deviations from mean
    bright_threshold = mean_val + (2.0 - sensitivity * 0.5) * std_val
    bright_mask = smoothed > bright_threshold
    
    # Clean up and remove borders
    bright_mask = binary_opening(bright_mask, disk(3))
    bright_mask = remove_border_regions(bright_mask, border_size=15)
    bright_mask = remove_small_objects(bright_mask, min_size=min_region_size)
    
    results['Intensity_Threshold'] = bright_mask
    
    # ===== METHOD 2: Texture-based (FFT High-pass) =====
    try:
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=20)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        
        # High texture regions
        hp_threshold = np.percentile(hp_img, 85 - sensitivity * 15)
        hp_mask = hp_img > hp_threshold
        
        # Must ALSO be bright (to avoid edge artifacts)
        hp_mask = np.logical_and(hp_mask, smoothed > mean_val)
        
        # Remove borders and clean
        hp_mask = remove_border_regions(hp_mask, border_size=20)
        hp_mask = binary_closing(hp_mask, disk(5))
        hp_mask = remove_small_objects(hp_mask, min_size=min_region_size)
        
        results['FFT_HighPass'] = hp_mask
    except:
        results['FFT_HighPass'] = np.zeros_like(image, dtype=bool)
    
    # ===== METHOD 3: Band-pass detection =====
    try:
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=10, r2=50)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        
        bp_threshold = np.percentile(bp_img, 80 - sensitivity * 10)
        bp_mask = bp_img > bp_threshold
        
        # Combine with intensity
        bp_mask = np.logical_and(bp_mask, smoothed > mean_val)
        
        # Remove borders and clean
        bp_mask = remove_border_regions(bp_mask, border_size=20)
        bp_mask = binary_closing(bp_mask, disk(5))
        bp_mask = remove_small_objects(bp_mask, min_size=min_region_size)
        
        results['FFT_BandPass'] = bp_mask
    except:
        results['FFT_BandPass'] = np.zeros_like(image, dtype=bool)
    
    # ===== METHOD 4: Otsu baseline =====
    thresh = threshold_otsu(smoothed)
    otsu_mask = smoothed > thresh
    otsu_mask = remove_border_regions(otsu_mask, border_size=15)
    otsu_mask = remove_small_objects(otsu_mask, min_size=min_region_size)
    results['Baseline_Otsu'] = otsu_mask
    
    # ===== METHOD 5: Combined with STRICT voting =====
    # Require agreement from at least 2 methods (stricter than before)
    vote_map = (results['Intensity_Threshold'].astype(float) + 
                results['FFT_HighPass'].astype(float) + 
                results['FFT_BandPass'].astype(float) +
                results['Baseline_Otsu'].astype(float) * 0.5)  # Weight Otsu less
    
    # Need strong agreement (at least 2 methods, or 1.5 at high sensitivity)
    vote_threshold = max(1.5, 2.5 - sensitivity)
    combined = vote_map >= vote_threshold
    
    # Final cleanup
    combined = remove_border_regions(combined, border_size=20)
    combined = binary_closing(combined, disk(4))
    combined = remove_small_objects(combined, min_size=min_region_size)
    
    # Filter out regions that are clearly normal tissue boundaries
    labeled = label(combined)
    filtered_mask = np.zeros_like(combined)
    
    regions = regionprops(labeled, intensity_image=image)
    for region in regions:
        # Size check
        if region.area < min_region_size or region.area > max_region_size:
            continue
        
        # Check if it's a normal tissue boundary
        if is_normal_tissue_boundary(region, image):
            continue
        
        # Check compactness (tumors are somewhat round/compact)
        circularity = 4 * np.pi * region.area / (region.perimeter ** 2 + 1e-8)
        if circularity < 0.15:  # Too elongated/thin
            continue
        
        # Passed all filters - add to final mask
        filtered_mask[labeled == region.label] = True
    
    results['Combined'] = filtered_mask
    
    return results

def analyze_detection(image, mask):
    """
    Analyze detected regions with stricter criteria.
    """
    h, w = image.shape
    total_pixels = h * w
    
    tumor_pixels = mask.sum()
    tumor_area_pct = (tumor_pixels / total_pixels) * 100
    
    # No detection if area is too small
    if tumor_pixels < total_pixels * 0.005:  # Less than 0.5%
        return {
            'detected': False,
            'area_percent': 0,
            'center': None,
            'confidence': 0
        }
    
    # Find center
    coords = np.where(mask)
    if len(coords[0]) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    center = (int(np.mean(coords[0])), int(np.mean(coords[1])))
    
    # Analyze regions
    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)
    
    if len(regions) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    largest = max(regions, key=lambda x: x.area)
    
    # Stricter confidence calculation
    # 1. Size score (prefer 2-15% of image)
    ideal_size = 8  # 8% is a typical tumor size
    size_diff = abs(tumor_area_pct - ideal_size)
    size_score = max(0, 1 - size_diff / 20)
    
    # 2. Shape score (tumors are somewhat round)
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 2, 1.0)
    
    # 3. Contrast score
    tumor_intensity = image[mask].mean()
    bg_intensity = image[~mask].mean() if (~mask).any() else 0
    contrast = abs(tumor_intensity - bg_intensity)
    contrast_score = min(contrast * 3, 1.0)
    
    # 4. Location score (penalize if too close to edges)
    cy, cx = center
    edge_dist = min(cy, h-cy, cx, w-cx)
    location_score = min(edge_dist / (min(h, w) * 0.1), 1.0)
    
    # Weighted confidence
    confidence = (size_score * 0.25 + 
                 shape_score * 0.25 + 
                 contrast_score * 0.3 + 
                 location_score * 0.2)
    
    # Only report detection if confidence > threshold
    if confidence < 0.3:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    return {
        'detected': True,
        'area_percent': tumor_area_pct,
        'center': center,
        'confidence': np.clip(confidence, 0, 1)
    }

# Initialize
image = None
mask = None
print('✅ Setup complete! Enhanced tumor detection system ready.')
print('   Features: Border removal, tissue boundary filtering, strict validation')

---
## 📁 Load Your Image

In [ ]:
# ========== OPTION A: UPLOAD YOUR OWN IMAGE ==========
print('Upload your medical image (MRI/CT scan):')

uploaded = files.upload()

if uploaded:
    uploaded_files = list(uploaded.keys())
    image = load_image_safe(uploaded_files[0])
    mask = None
    
    print(f'\n✅ Loaded: {uploaded_files[0]}')
    print(f'   Size: {image.shape[1]} x {image.shape[0]} pixels')
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray')
    plt.title('Your Uploaded Image')
    plt.axis('off')
    plt.show()
else:
    print('❌ No file uploaded.')

In [ ]:
# ========== OPTION B: USE SAMPLE IMAGES ==========
tumor_number = 1  # Change to 1, 2, or 3

tumor_file = f'tumor_{tumor_number:03d}.png'
print(f'Loading: {tumor_file}')

image = load_image_safe(f'data/images/{tumor_file}')
mask = load_mask_safe(f'data/masks/{tumor_file}')

print(f'✅ Loaded tumor {tumor_number}')

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(image, cmap='gray')
axes[0].set_title('Sample Image')
axes[0].axis('off')
axes[1].imshow(image, cmap='gray')
axes[1].imshow(mask, cmap='Reds', alpha=0.5)
axes[1].set_title('Ground Truth')
axes[1].axis('off')
plt.show()

---
## 🔬 FFT Analysis

In [ ]:
# ========== FFT SPECTRUM ==========
if image is not None:
    F_shift, mag = ts.compute_fft_spectrum(image)
    fig = ts.visualize_frequency_spectrum(image, F_shift)
    plt.show()
else:
    print('⚠️ No image loaded!')

---
## 🎯 TUMOR DETECTION

In [ ]:
# ========== METHOD COMPARISON ==========
if image is not None:
    print('Running enhanced tumor detection...\n')
    
    detection_results = detect_tumor_regions(image, sensitivity=0.5)
    
    methods = ['Baseline_Otsu', 'Intensity_Threshold', 'FFT_HighPass', 'FFT_BandPass', 'Combined']
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    for idx, method in enumerate(methods):
        detected_mask = detection_results[method]
        analysis = analyze_detection(image, detected_mask)
        
        axes[idx+1].imshow(image, cmap='gray')
        axes[idx+1].imshow(detected_mask, cmap='Reds', alpha=0.6)
        
        if analysis['center']:
            row, col = analysis['center']
            axes[idx+1].plot(col, row, 'g+', markersize=15, markeredgewidth=2)
        
        status = f"Area: {analysis['area_percent']:.1f}%" if analysis['detected'] else "Not detected"
        axes[idx+1].set_title(f'{method}\n{status}', fontsize=11, fontweight='bold')
        axes[idx+1].axis('off')
    
    plt.suptitle('Tumor Detection with Border/Artifact Filtering', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary
    print('\n' + '='*70)
    print(f'{"Method":<20} {"Detected":<12} {"Area %":<12} {"Confidence":<12}')
    print('='*70)
    for method in methods:
        analysis = analyze_detection(image, detection_results[method])
        det = '✓ YES' if analysis['detected'] else '✗ No'
        area = f"{analysis['area_percent']:.2f}%"
        conf = f"{analysis['confidence']*100:.0f}%" if analysis['detected'] else '-'
        print(f'{method:<20} {det:<12} {area:<12} {conf:<12}')
    print('='*70)
else:
    print('⚠️ No image loaded!')

---
## 📋 FINAL DETECTION VERDICT

In [ ]:
# ========== FINAL RESULT ==========
if image is not None:
    final_mask = detection_results['Combined']
    final_analysis = analyze_detection(image, final_mask)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    if final_analysis['detected']:
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.6)
        if final_analysis['center']:
            row, col = final_analysis['center']
            axes[1].plot(col, row, 'g+', markersize=25, markeredgewidth=3)
    axes[1].set_title('Detected Tumor Region', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].axis('off')
    
    if final_analysis['detected']:
        if final_analysis['confidence'] > 0.6:
            status = '⚠️ TUMOR DETECTED'
            verdict = '🔴 HIGH CONFIDENCE'
        elif final_analysis['confidence'] > 0.35:
            status = '❓ POSSIBLE TUMOR'
            verdict = '🟡 MEDIUM CONFIDENCE'
        else:
            status = '❓ SUSPICIOUS REGION'
            verdict = '🟠 LOW CONFIDENCE'
    else:
        status = '✅ NO TUMOR DETECTED'
        verdict = '🟢 Tissue appears normal'
    
    results_text = f"""
    
    {status}
    
    ─────────────────────────────
    
    Confidence: {final_analysis['confidence']*100:.1f}%
    
    Tumor Area: {final_analysis['area_percent']:.2f}%
    
    {verdict}
    
    """
    
    axes[2].text(0.1, 0.5, results_text, fontsize=14, verticalalignment='center',
                fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    axes[2].set_title('Detection Results', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*50)
    print(f'  {verdict}')
    print('='*50)
else:
    print('⚠️ No image loaded!')

---
## 📊 Validation (if ground truth available)

In [ ]:
# ========== VALIDATION ==========
if image is not None and mask is not None:
    detected = detection_results['Combined']
    
    intersection = np.logical_and(detected, mask).sum()
    union = np.logical_or(detected, mask).sum()
    dice = 2 * intersection / (detected.sum() + mask.sum() + 1e-8)
    iou = intersection / (union + 1e-8)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].imshow(mask, cmap='Greens', alpha=0.5)
    axes[0].set_title('Ground Truth', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    axes[1].imshow(detected, cmap='Reds', alpha=0.5)
    axes[1].set_title('Our Detection', fontweight='bold')
    axes[1].axis('off')
    
    overlap = np.zeros((*image.shape, 3))
    overlap[mask > 0] = [0, 1, 0]
    overlap[detected > 0] = [1, 0, 0]
    overlap[np.logical_and(mask, detected)] = [1, 1, 0]
    
    axes[2].imshow(image, cmap='gray')
    axes[2].imshow(overlap, alpha=0.6)
    axes[2].set_title(f'Overlap (Dice: {dice:.3f})', fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f'\nDice Score: {dice:.3f}')
    print(f'IoU Score: {iou:.3f}')
elif mask is None:
    print('ℹ️ No ground truth available for validation')
else:
    print('⚠️ No image loaded!')